In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
%matplotlib inline

In [2]:
cd = "C:\\Users\\ASUS\\Desktop\\bias\\all_datasets"

In [3]:
os.listdir(cd)

['crsp_msf.csv',
 'ibes_unadjusted_actuals.csv',
 'ibes_unadjusted_summary.csv',
 'IPT_1962-M11_present.xlsx',
 'real_GDP_1965-M11_present.xlsx',
 'real_GDP_1965-Q4_present.xlsx',
 'real_personal_consumption_1965-M11_present.xlsx',
 'real_personal_consumption_1965-Q4_present.xlsx',
 'unemployment_1965-Q4_present.xlsx',
 'wrdscomp_finratios.csv',
 'wrds_ibes_crsp_links.csv',
 '~$real_GDP_1965-M11_present.xlsx',
 '~$real_personal_consumption_1965-M11_present.xlsx']

In [4]:
os.chdir(cd)

### CRSP msf
- relevant variables: `PRC`, `RET`, `CFACSHR`
- fix delisting return: `msf.RET.update(msf.DLRET)`
- If the closing price is not available on any given trading day, the number in the price field has a negative sign to indicate that it is a bid/ask average and not an actual closing price. Please note that in this field the negative sign is a symbol and that the value of the bid/ask average is not negative.

In [5]:
CRSP = pd.read_csv('crsp_msf.csv',usecols=['PERMNO', 'date', 'SHRCD', 'EXCHCD', 'RET', 'DLRET', 'PRC', 'CFACPR', 'CFACSHR'], low_memory=False)

In [6]:
CRSP = CRSP[(CRSP.SHRCD.apply(lambda x: x in [10.0, 11.0])) & (CRSP.EXCHCD.apply(lambda x: x in [1, 2, 3]))].reset_index(drop=True)

In [7]:
CRSP.PRC = abs(CRSP['PRC'])
CRSP.loc[CRSP.DLRET.notnull(), 'RET'] = CRSP.DLRET

In [8]:
CRSP['rankdate']=pd.to_datetime(CRSP.date,format='%Y%m%d')
CRSP['rankdate'] = CRSP['rankdate'].dt.to_period('M')
CRSP['RET'] = pd.to_numeric(CRSP['RET'], errors='coerce')

In [9]:
# 观察到RET序列中有字母，因此需提前删除
# CRSP = CRSP.sort_values(by=['PERMNO','rankdate'], ascending=True)
# 考虑不删除缺失值
CRSP = CRSP.sort_values(by=['PERMNO','rankdate'], ascending=True)

### IBES and IBES_CRSP_link
- relevant identifiers: `TICKER` for IBES, `CUSIP`, `PERMNO` for CRSP
- forecast time horizons: `FPI` in (1,2,6,7,8) --> 1-year, 2-year, one-quarter, two-quarters, three-quarters
- match IBES actual file (actual realized earnings) with summary file (analysts' consensus forecasts) using Ticker and fiscal end date (`FPEDATS` in summary and `PENDS` in actual)
- match IBES with CRSP: use `SCORE`=1, filter `SHRCD` in (10,11), `EXCHCD` in (1,2,3) --> historical cusip, common stocks, major exchanges

In [10]:
summary_file = pd.read_csv('ibes_unadjusted_summary.csv', usecols=['TICKER', 'CUSIP', 'STATPERS', 'FPI', 'MEANEST', 'FPEDATS'])

In [11]:
summary_file = summary_file[summary_file.FPI != 9]

In [12]:
summary_file.head()

,TICKER,CUSIP,STATPERS,FPI,MEANEST,FPEDATS
0,0000,87482X10,2014-04-17,6,0.08,2014-03-31
1,0000,87482X10,2014-05-15,6,0.13,2014-06-30
2,0000,87482X10,2014-06-19,6,0.13,2014-06-30
3,0000,87482X10,2014-07-17,6,0.13,2014-06-30
4,0000,87482X10,2014-04-17,7,0.12,2014-06-30


In [13]:
actual = pd.read_csv('ibes_unadjusted_actuals.csv', usecols=['TICKER', 'PENDS', 'VALUE', 'PDICITY'])

In [14]:
actual = actual[actual.PDICITY == 'ANN'].drop('PDICITY', axis=1).dropna(subset=['TICKER', 'VALUE'], how='any').sort_values(by=['TICKER', 'PENDS'], ascending=True)

In [15]:
actual['past_eps'] = actual.groupby('TICKER').apply(lambda stock: stock.VALUE.shift(1)).values

In [16]:
IBES = pd.merge(actual, summary_file, how='inner', left_on=['TICKER', 'PENDS'], right_on=['TICKER', 'FPEDATS']).drop('FPEDATS', axis=1)

In [17]:
IBES.head()

,TICKER,PENDS,VALUE,past_eps,CUSIP,STATPERS,FPI,MEANEST
0,0000,2014-12-31,1.21,NaN,87482X10,2014-04-17,1,0.52
1,0000,2014-12-31,1.21,NaN,87482X10,2014-05-15,1,0.56
2,0000,2014-12-31,1.21,NaN,87482X10,2014-06-19,1,0.56
3,0000,2014-12-31,1.21,NaN,87482X10,2014-07-17,1,0.56
4,0000,2014-12-31,1.21,NaN,87482X10,2014-08-14,1,1.18


In [18]:
IBES.shape

(12122826, 8)

In [19]:
link_table = pd.read_csv('wrds_ibes_crsp_links.csv', usecols=['PERMNO', 'NCUSIP', 'SCORE'])

In [20]:
link_table.head()

,PERMNO,NCUSIP,SCORE
0,14471.0,87482X10,1
1,14392.0,26878510,1
2,NaN,NaN,6
3,14418.0,02504D10,1
4,14378.0,14163310,1


In [21]:
link_table = link_table[link_table.SCORE == 1].drop('SCORE', axis=1)

In [22]:
IBES_link = pd.merge(IBES, link_table, how='inner', left_on=['CUSIP'], right_on=['NCUSIP']).drop('NCUSIP', axis=1)

In [23]:
IBES_link.head()

,TICKER,PENDS,VALUE,past_eps,CUSIP,STATPERS,FPI,MEANEST,PERMNO
0,0000,2014-12-31,1.21,NaN,87482X10,2014-04-17,1,0.52,14471.0
1,0000,2014-12-31,1.21,NaN,87482X10,2014-05-15,1,0.56,14471.0
2,0000,2014-12-31,1.21,NaN,87482X10,2014-06-19,1,0.56,14471.0
3,0000,2014-12-31,1.21,NaN,87482X10,2014-07-17,1,0.56,14471.0
4,0000,2014-12-31,1.21,NaN,87482X10,2014-08-14,1,1.18,14471.0


In [24]:
IBES_link['rankdate'] = pd.to_datetime(IBES_link.STATPERS).dt.to_period('M')

In [25]:
IBES_link['PERMNO'] = IBES_link['PERMNO'].astype('int')

In [26]:
IBES_CRSP = pd.merge(CRSP[['PERMNO', 'PRC', 'RET', 'CFACPR', 'CFACSHR', 'rankdate']], IBES_link, how='inner',  on=['PERMNO', 'rankdate'])

In [27]:
IBES_CRSP.head()

,PERMNO,PRC,RET,CFACPR,CFACSHR,rankdate,TICKER,PENDS,VALUE,past_eps,CUSIP,STATPERS,FPI,MEANEST
0,10001,6.75,0.017778,3.0,3.0,1989-03,GFGC,1989-06-30,1.21,0.55,39040610,1989-03-16,1,0.6
1,10001,7.25,0.074074,3.0,3.0,1989-04,GFGC,1989-06-30,1.21,0.55,39040610,1989-04-20,1,0.6
2,10001,7.00,-0.034483,3.0,3.0,1989-05,GFGC,1989-06-30,1.21,0.55,39040610,1989-05-18,1,0.6
3,10001,7.00,0.017143,3.0,3.0,1989-06,GFGC,1989-06-30,1.21,0.55,39040610,1989-06-15,1,0.6
4,10001,7.25,0.035714,3.0,3.0,1989-07,GFGC,1989-06-30,1.21,0.55,39040610,1989-07-20,1,0.6


### Macroeconomic variables:
- real consumption
- unemployment rate
- real GDP
- Industrial Production Index


In [33]:
ipi = pd.read_excel('IPT_1962-M11_present.xlsx')
rgdp = pd.read_excel('real_GDP_1965-M11_present.xlsx')
rpc = pd.read_excel('real_personal_consumption_1965-M11_present.xlsx')
unemp = pd.read_excel('unemployment_1965-Q4_present.xlsx')

In [30]:
ipi['DATE'] = pd.to_datetime(pd.Series(list(map(lambda date: datetime.strptime(date, '%Y:%m'), ipi.DATE)))).dt.to_period('M')
ipi.set_index('DATE', drop=True, inplace=True)
ipi = np.log(ipi).apply(lambda x: x - x.shift(1), axis=0)

In [28]:
ipi

,IPT62M11,IPT62M12,IPT63M1,IPT63M2,IPT63M3,IPT63M4,IPT63M5,IPT63M6,IPT63M7,IPT63M8,...,IPT22M5,IPT22M6,IPT22M7,IPT22M8,IPT22M9,IPT22M10,IPT22M11,IPT22M12,IPT23M1,IPT23M2
DATE,,,,,,,,,,,,,,,,,,,,,
1919-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.655863,4.652054,4.652054,4.652054,4.650144
2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.651099,4.651099,4.652054,4.650144
2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.649187,4.646312,4.644391


In [33]:
# ipi_date = pd.to_datetime(pd.Series(pd.date_range(start='19621130', end='20230228', freq='M'))).dt.to_period('M')

In [34]:
# series = list(map(lambda x: str(int(x)), np.linspace(24, 1, 24, endpoint=True)))
# ipi_series = ['ipi_' + i for i in series]

In [35]:
# IPI = pd.DataFrame({}, index=ipi_date, columns=ipi_series)

In [37]:
# 找出观测年份之前两年的观测值
# col = 1
# for date in IPI.index:
#     try:
#         row = ipi[ipi.DATE == date].index.tolist()[0]
#         IPI.loc[date] = ipi.iloc[(row - 24):row, col].values
#     except Exception as e:
#         IPI.loc[date] = ipi.iloc[(len(ipi) - 24):, col].values
#     col += 1

In [42]:
rgdp

,DATE,ROUTPUT65M11,ROUTPUT65M12,ROUTPUT66M1,ROUTPUT66M2,ROUTPUT66M3,ROUTPUT66M4,ROUTPUT66M5,ROUTPUT66M6,ROUTPUT66M7,...,ROUTPUT22M6,ROUTPUT22M7,ROUTPUT22M8,ROUTPUT22M9,ROUTPUT22M10,ROUTPUT22M11,ROUTPUT22M12,ROUTPUT23M1,ROUTPUT23M2,ROUTPUT23M3
0,1947:Q1,306.4,306.4,306.4,306.4,306.4,306.4,306.4,306.4,306.4,...,2034.5,2034.5,2034.5,2034.5,2034.5,2034.5,2034.5,2034.5,2034.5,2034.5
1,1947:Q2,309.0,309.0,309.0,309.0,309.0,309.0,309.0,309.0,309.0,...,2029.0,2029.0,2029.0,2029.0,2029.0,2029.0,2029.0,2029.0,2029.0,2029.0
2,1947:Q3,309.6,309.6,309.6,309.6,309.6,309.6,309.6,309.6,309.6,...,2024.8,2024.8,2024.8,2024.8,2024.8,2024.8,2024.8,2024.8,2024.8,2024.8
3,1947:Q4,314.5,314.5,314.5,314.5,314.5,314.5,314.5,314.5,314.5,...,2056.5,2056.5,2056.5,2056.5,2056.5,2056.5,2056.5,2056.5,2056.5,2056.5
4,1948:Q1,317.1,317.1,317.1,317.1,317.1,317.1,317.1,317.1,317.1,...,2087.4,2087.4,2087.4,2087.4,2087.4,2087.4,2087.4,2087.4,2087.4,2087.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,2021:Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19806.3,19806.3,19806.3,19806.3,20006.2,20006.2,20006.2,20006.2,20006.2,20006.2
300,2022:Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19731.1,19727.9,19727.9,19727.9,19924.1,19924.1,19924.1,19924.1,19924.1,19924.1
301,2022:Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19681.7,19699.5,19895.3,19895.3,19895.3,19895.3,19895.3,19895.3
302,2022:Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20021.7,20039.4,20054.7,20054.7,20054.7


In [31]:
rgdp.drop('DATE', axis=1, inplace=True)

In [32]:
rgdp = np.log(rgdp).apply(lambda x: x - x.shift(1), axis=0)

In [33]:
real_gdp = pd.DataFrame(np.repeat(rgdp.values, 3, axis=0))
real_gdp.columns = rgdp.columns

In [34]:
real_gdp.index = pd.to_datetime(pd.Series(pd.date_range(start='19470228', end='20230131', freq='M'))).dt.to_period('M')
real_gdp.index.name = 'DATE'

In [35]:
real_gdp

,ROUTPUT65M11,ROUTPUT65M12,ROUTPUT66M1,ROUTPUT66M2,ROUTPUT66M3,ROUTPUT66M4,ROUTPUT66M5,ROUTPUT66M6,ROUTPUT66M7,ROUTPUT66M8,...,ROUTPUT22M6,ROUTPUT22M7,ROUTPUT22M8,ROUTPUT22M9,ROUTPUT22M10,ROUTPUT22M11,ROUTPUT22M12,ROUTPUT23M1,ROUTPUT23M2,ROUTPUT23M3
DATE,,,,,,,,,,,,,,,,,,,,,
1947-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-05,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,...,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707
1947-06,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,0.00845,...,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707,-0.002707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.006333,0.007217,0.007980,0.007980,0.007980
2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.006333,0.007217,0.007980,0.007980,0.007980
2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007125,0.006600


In [ ]:
# rgdp['DATE'] = pd.to_datetime(pd.Series(list(map(lambda date: datetime.strptime(date, '%Y:%m'), rgdp.DATE)))).dt.to_period('M')

In [39]:
# rgdp_date = pd.to_datetime(pd.Series(pd.date_range(start='19651130', end='20230131', freq='M'))).dt.to_period('M')
# series = list(map(lambda x: str(int(x)), np.linspace(8, 1, 8, endpoint=True)))
# rgdp_series = ['rgdp_' + i for i in series]

In [40]:
# RGDP = pd.DataFrame({}, index=rgdp_date, columns=rgdp_series)

In [41]:
# RGDP

,rgdp_8,rgdp_7,rgdp_6,rgdp_5,rgdp_4,rgdp_3,rgdp_2,rgdp_1
1965-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1965-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1966-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1966-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1966-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# The quarterly vintages are taken from the monthly vintages for February, May, August, and November
# col = 1
# for date in RGDP.index:
#     quarter = str(date.year)
#     if date.month in [2, 3, 4]:
#         quarter = str(date.year) + ':Q1'
#     elif date.month in [5, 6, 7]:
#         quarter = str(date.year) + ':Q2'
#     elif date.month in [8, 9, 10]:
#         quarter = str(date.year) + ':Q3'
#     elif date.month == 1:
#         quarter = str(date.year - 1) + ':Q4'
#     else:
#         quarter = str(date.year) + ':Q4'
#     try:
#         row = rgdp[rgdp.DATE == quarter].index.tolist()[0]
#         RGDP.loc[date] = rgdp.iloc[(row - 8):row, col].values
#     except Exception as e:
#         RGDP.loc[date] = rgdp.iloc[(len(rgdp) - 8):, col].values
#     col += 1

In [46]:
rpc

,DATE,RCON65M11,RCON65M12,RCON66M1,RCON66M2,RCON66M3,RCON66M4,RCON66M5,RCON66M6,RCON66M7,...,RCON22M6,RCON22M7,RCON22M8,RCON22M9,RCON22M10,RCON22M11,RCON22M12,RCON23M1,RCON23M2,RCON23M3
0,1947:Q1,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,...,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4
1,1947:Q2,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,...,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7
2,1947:Q3,207.4,207.4,207.4,207.4,207.4,207.4,207.4,207.4,207.4,...,1302.9,1302.9,1302.9,1302.9,1302.9,1302.9,1302.9,1302.9,1302.9,1302.9
3,1947:Q4,207.3,207.3,207.3,207.3,207.3,207.3,207.3,207.3,207.3,...,1303.3,1303.3,1303.3,1303.3,1303.3,1303.3,1303.3,1303.3,1303.3,1303.3
4,1948:Q1,208.5,208.5,208.5,208.5,208.5,208.5,208.5,208.5,208.5,...,1309.8,1309.8,1309.8,1309.8,1309.8,1309.8,1309.8,1309.8,1309.8,1309.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,2021:Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13818.4,13818.4,13818.4,13818.4,13981.5,13981.5,13981.5,13981.5,13981.5,13981.5
300,2022:Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13924.8,13881.1,13881.1,13881.1,14028.4,14028.4,14028.4,14028.4,14028.4,14028.4
301,2022:Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,13916.9,13931.5,14099.5,14099.5,14099.5,14099.5,14099.5,14099.5
302,2022:Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14149.0,14159.9,14178.6,14178.6,14178.6


In [36]:
rpc.drop('DATE', axis=1, inplace=True)

In [37]:
rpc = np.log(rpc).apply(lambda x: x - x.shift(1), axis=0)

In [38]:
real_pc = pd.DataFrame(np.repeat(rpc.values, 3, axis=0))
real_pc.columns = rpc.columns

In [39]:
real_pc.index = pd.to_datetime(pd.Series(pd.date_range(start='19470228', end='20230131', freq='M'))).dt.to_period('M')
real_pc.index.name = 'DATE'

In [50]:
real_pc

,RCON65M11,RCON65M12,RCON66M1,RCON66M2,RCON66M3,RCON66M4,RCON66M5,RCON66M6,RCON66M7,RCON66M8,...,RCON22M6,RCON22M7,RCON22M8,RCON22M9,RCON22M10,RCON22M11,RCON22M12,RCON23M1,RCON23M2,RCON23M3
1947-02,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,...,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4
1947-03,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,...,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4
1947-04,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,203.4,...,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4,1277.4
1947-05,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,...,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7
1947-06,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,207.0,...,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7,1298.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14149.0,14159.9,14178.6,14178.6,14178.6
2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14149.0,14159.9,14178.6,14178.6,14178.6
2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14252.2,14226.8
2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14252.2,14226.8


In [44]:
# rpc_date = pd.to_datetime(pd.Series(pd.date_range(start='19651130', end='20230331', freq='M'))).dt.to_period('M')
# rpc_series = ['rpc_' + i for i in series]

In [45]:
# RPC = pd.DataFrame({}, index=rpc_date, columns=rpc_series)

In [47]:
# The quarterly vintages are taken from the monthly vintages for February, May, August, and November
# col = 1
# for date in RPC.index:
#     if date.month in [2, 3, 4]:
#         quarter = str(date.year) + ':Q1'
#     elif date.month in [5, 6, 7]:
#         quarter = str(date.year) + ':Q2'
#     elif date.month in [8, 9, 10]:
#         quarter = str(date.year) + ':Q3'
#     elif date.month == 1:
#         quarter = str(date.year - 1) + ':Q4'
#     else:
#         quarter = str(date.year) + ':Q4'
#     try:
#         row = rpc[rpc.DATE == quarter].index.tolist()[0]
#         RPC.loc[date] = rpc.iloc[(row - 8):row, col].values
#     except Exception as e:
#         RPC.loc[date] = rpc.iloc[(len(rpc) - 8):, col].values
#     col += 1

In [42]:
unemp

,RUC65Q4,RUC66Q1,RUC66Q2,RUC66Q3,RUC66Q4,RUC67Q1,RUC67Q2,RUC67Q3,RUC67Q4,RUC68Q1,...,RUC20Q4,RUC21Q1,RUC21Q2,RUC21Q3,RUC21Q4,RUC22Q1,RUC22Q2,RUC22Q3,RUC22Q4,RUC23Q1
DATE,,,,,,,,,,,,,,,,,,,,,
1947-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.5
2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7,3.7
2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6


In [48]:
# unemp_date = pd.to_datetime(pd.Series(pd.date_range(start='19651130', end='20230331', freq='M'))).dt.to_period('M')
# series = list(map(lambda x: str(int(x)), np.linspace(24, 1, 24, endpoint=True)))
# unemp_series = ['unemp_' + i for i in series]

In [40]:
unemp['DATE'] = pd.to_datetime(pd.Series(list(map(lambda x: datetime.strptime(x, '%Y:%m'), unemp.DATE)))).dt.to_period('M')

In [41]:
unemp.set_index('DATE', drop=True, inplace=True)

In [50]:
# UNEMP = pd.DataFrame({}, index=unemp_date, columns=unemp_series)

In [52]:
# for date in UNEMP.index:
#     d_origin = date
#     if date.month in [2, 3, 4]:
#         col = 'RUC' + str(date.year)[-2:] + 'Q1'
#         adjust = date.month - 2
#     elif date.month in [5, 6, 7]:
#         col = 'RUC' + str(date.year)[-2:] + 'Q2'
#         adjust = date.month - 5
#     elif date.month in [8, 9, 10]:
#         col = 'RUC' + str(date.year)[-2:] + 'Q3'
#         adjust = date.month - 8
#     elif date.month == 1:
#         col = 'RUC' + str(date.year - 1)[-2:] + 'Q4'
#         adjust = 2
#     else:
#         col = 'RUC' + str(date.year)[-2:] + 'Q4'
#         adjust = date.month - 11
#     try:
#         row = unemp[unemp.DATE == date].index.tolist()[0] - adjust
#         UNEMP.loc[d_origin] = unemp.loc[(row - 24):(row - 1), col].values
#     except Exception as e:
#         UNEMP.loc[d_origin] = unemp.loc[(len(unemp) - 24):, col].values

In [43]:
ipi.to_csv("C:\\Users\\ASUS\\Desktop\\data\\ipi.csv")
real_gdp.to_csv("C:\\Users\\ASUS\\Desktop\\data\\rgdp.csv")
real_pc.to_csv("C:\\Users\\ASUS\\Desktop\\data\\rpc.csv")
unemp.to_csv("C:\\Users\\ASUS\\Desktop\\data\\unemp.csv")

### WRDS financial ratios
- relevant identifiers: `gvkey`, `permno`
- relevant dates: `adate`, `qdate` stand for annual and quarterly dates --> annual and quarterly frequencies
- compare with __Table A1: WRDS financial ratios__ for corresponding variable names

In [28]:
fin_ratio = pd.read_csv('wrdscomp_finratios.csv', low_memory=False)

In [29]:
fin_ratio.head()

,gvkey,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward,TICKER,cusip
0,1000,25881,19691231.0,19700930.0,19710131,10.950,0.427,1.761,NaN,NaN,...,0.0,0.0,0.530,2.340,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,25881,19701231.0,19701231.0,19710228,35.407,0.431,7.010,NaN,NaN,...,0.0,0.0,0.021,2.801,NaN,NaN,NaN,NaN,AE,00003210
2,1000,25881,19701231.0,19701231.0,19710331,31.031,0.431,7.010,NaN,NaN,...,0.0,0.0,0.021,2.455,NaN,NaN,NaN,NaN,AE,00003210
3,1000,25881,19701231.0,19701231.0,19710430,34.214,0.431,7.010,NaN,NaN,...,0.0,0.0,0.021,2.707,NaN,NaN,NaN,NaN,AE,00003210
4,1000,25881,19701231.0,19710331.0,19710531,11.307,0.431,7.010,NaN,NaN,...,0.0,0.0,0.021,2.172,NaN,NaN,NaN,NaN,AE,00003210


In [30]:
fin_ratio.rename(columns={'permno': 'PERMNO'}, inplace=True)

In [31]:
fin_ratio['rankdate'] = pd.to_datetime(fin_ratio['public_date'].astype('str')).dt.to_period('M')

In [32]:
fin_ratio.drop(['gvkey', 'adate', 'qdate', 'public_date', 'TICKER', 'cusip'], axis=1, inplace=True)

In [50]:
fin_ratio.shape

(2921483, 73)

In [33]:
data = pd.merge(IBES_CRSP, fin_ratio, how='inner', on=['PERMNO', 'rankdate'])

### Generate the whole dataset separated by the FPI

In [34]:
# divyield列是str格式，需要先转换
data['divyield'] = data['divyield'].str.replace('%', '').astype('float')

In [35]:
A1_RF = data[data.FPI == 1].reset_index(drop=True)
A2_RF = data[data.FPI == 2].reset_index(drop=True)
Q1_RF = data[data.FPI == 6].reset_index(drop=True)
Q2_RF = data[data.FPI == 7].reset_index(drop=True)
Q3_RF = data[data.FPI == 8].reset_index(drop=True)

In [42]:
A1_RF

,PERMNO,PRC,RET,CFACPR,CFACSHR,rankdate,TICKER,PENDS,VALUE,past_eps,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10001,6.75000,0.017778,3.0,3.0,1989-03,GFGC,1989-06-30,1.21,0.55,...,152.923,0.000,0.0,0.0,0.027,0.873,0.018,7.11,0.274,NaN
1,10001,7.25000,0.074074,3.0,3.0,1989-04,GFGC,1989-06-30,1.21,0.55,...,152.923,0.000,0.0,0.0,0.027,0.937,0.020,6.62,0.295,NaN
2,10001,7.00000,-0.034483,3.0,3.0,1989-05,GFGC,1989-06-30,1.21,0.55,...,39.029,0.000,0.0,0.0,-0.015,0.842,0.025,6.86,0.251,NaN
3,10001,7.00000,0.017143,3.0,3.0,1989-06,GFGC,1989-06-30,1.21,0.55,...,39.029,0.000,0.0,0.0,-0.015,0.845,0.025,6.86,0.251,NaN
4,10001,7.25000,0.035714,3.0,3.0,1989-07,GFGC,1989-06-30,1.21,0.55,...,39.029,0.000,0.0,0.0,-0.015,0.875,0.026,6.62,0.260,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435618,93436,775.47998,0.054042,3.0,3.0,2021-09,TSLA,2021-12-31,6.78,2.24,...,4.287,0.051,0.0,0.0,-0.143,31.310,NaN,NaN,0.982,3.828
1435619,93436,1114.00000,0.436530,3.0,3.0,2021-10,TSLA,2021-12-31,6.78,2.24,...,4.287,0.051,0.0,0.0,-0.143,44.989,NaN,NaN,1.321,5.274
1435620,93436,1144.76001,0.027612,3.0,3.0,2021-11,TSLA,2021-12-31,6.78,2.24,...,4.988,0.051,0.0,0.0,-0.123,42.433,1.107,NaN,0.727,1.696
1435621,93436,1056.78003,-0.076855,3.0,3.0,2021-12,TSLA,2021-12-31,6.78,2.24,...,4.988,0.051,0.0,0.0,-0.123,40.313,1.022,NaN,0.671,1.565


In [36]:

for horizon in ('A1_', 'A2_', 'Q1_', 'Q2_', 'Q3_'):
    Forecast = eval(horizon + 'RF')
    Forecast['adj_actual'] = Forecast.groupby('PERMNO', group_keys=True).apply(lambda x: x['VALUE'] * x['CFACSHR'].shift(1) / x['CFACSHR']).values
    Forecast.dropna(subset='adj_actual', inplace=True)
    # Forecast.sort_values(by=['rankdate'], ascending=True, inplace=True)
    # for predictor in predictors:
        # Forecast[predictor] = Forecast.groupby('rankdate', group_keys=True).apply(lambda df: df[predictor].fillna(df[predictor].median()))
        # Forecast[predictor] = Forecast.groupby('rankdate', group_keys=True)[predictor].apply(winsorization).values
    Forecast.sort_values(by=['PERMNO', 'rankdate'], ascending=True, inplace=True)
    Forecast.reset_index(drop=True, inplace=True)
    Forecast.to_csv(f"C:\\Users\\ASUS\\Desktop\\data\\{horizon}.csv", index=False)